# Robot Base Floor
<img src="../Kaggle/CareerCon 2019 - Help Navigate Robots/pic.jpg" />

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, confusion_matrix
from bayes_opt import BayesianOptimization
%matplotlib inline

# Import Data

In [2]:
X_train =pd.read_csv('.\\Kaggle\\CareerCon 2019 - Help Navigate Robots\\data\\X_train.csv')
Y_train =pd.read_csv('.\\Kaggle\\CareerCon 2019 - Help Navigate Robots\\data\\Y_train.csv')
X_test =pd.read_csv('.\\Kaggle\\CareerCon 2019 - Help Navigate Robots\\data\\X_test.csv')
Submission=pd.read_csv('.\\Kaggle\\CareerCon 2019 - Help Navigate Robots\\data\\sample_submission.csv')

In [3]:
def filter_signal(signal, threshold=1e3):
    fourier = np.fft.rfft(signal)
    frequencies = np.fft.rfftfreq(signal.size, d=20e-3/signal.size)
    fourier[frequencies > threshold] = 0
    return np.fft.irfft(fourier)
X_train_denoised = X_train.copy()
X_test_denoised = X_test.copy()
# train
for col in X_train.columns:
    if col[0:3] == 'ang' or col[0:3] == 'lin':
        # Apply filter_signal function to the data in each series
        denoised_data = X_train.groupby(['series_id'])[col].apply(lambda x: filter_signal(x))
        
        # Assign the denoised data back to X_train
        list_denoised_data = []
        for arr in denoised_data:
            for val in arr:
                list_denoised_data.append(val)
                
        X_train_denoised[col] = list_denoised_data
X_train_denoised       
# test
for col in X_test.columns:
    if col[0:3] == 'ang' or col[0:3] == 'lin':
        # Apply filter_signal function to the data in each series
        denoised_data = X_test.groupby(['series_id'])[col].apply(lambda x: filter_signal(x))
        
        # Assign the denoised data back to X_train
        list_denoised_data = []
        for arr in denoised_data:
            for val in arr:
                list_denoised_data.append(val)
                
        X_test_denoised[col] = list_denoised_data
        

In [4]:
for col in X_train_denoised.columns:
    if col[0:3] == 'ang' or col[0:3] == 'lin':
        X_train_denoised[col+'_Amp']=abs(np.fft.fft(X_train_denoised[col]))*2/X_train_denoised.shape[0]
        X_train_denoised[col+'_Phase']=np.arctan2(np.fft.fft(X_train_denoised[col]).imag, np.fft.fft(X_train_denoised[col]).real) 
X_train_denoised.head()

,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,...,angular_velocity_Y_Amp,angular_velocity_Y_Phase,angular_velocity_Z_Amp,angular_velocity_Z_Phase,linear_acceleration_X_Amp,linear_acceleration_X_Phase,linear_acceleration_Y_Amp,linear_acceleration_Y_Phase,linear_acceleration_Z_Amp,linear_acceleration_Z_Phase
0,0_0,0,0,-0.75853,-0.63435,-0.10488,-0.10597,0.064074,0.017639,0.005879,...,0.016675,0.000000,0.038367,3.141593,0.258562,0.000000,5.772935,0.000000,18.729771,3.141593
1,0_1,0,1,-0.75853,-0.63434,-0.10490,-0.10600,0.055530,0.026568,-0.000263,...,0.002681,-1.613060,0.009702,1.575424,0.003197,-0.246583,0.032862,-1.197256,0.008725,-1.269914
2,0_2,0,2,-0.75853,-0.63435,-0.10492,-0.10597,0.033545,0.026603,-0.003403,...,0.001653,-2.996908,0.004939,0.256185,0.005468,-1.900897,0.004532,0.523438,0.001650,2.420690
3,0_3,0,3,-0.75852,-0.63436,-0.10495,-0.10597,0.020209,0.020964,-0.004373,...,0.002491,-0.199721,0.009018,2.903242,0.005125,-2.158967,0.010557,-1.711563,0.005018,-1.351602
4,0_4,0,4,-0.75852,-0.63435,-0.10495,-0.10596,0.025990,0.016635,-0.005075,...,0.001555,1.651569,0.004734,-1.332085,0.003560,1.420007,0.001038,1.302078,0.001414,2.224966


In [5]:
for col in X_test_denoised.columns:
    if col[0:3] == 'ang' or col[0:3] == 'lin':
        X_test_denoised[col+'_Amp']=abs(np.fft.fft(X_test_denoised[col]))*2/X_test_denoised.shape[0]
        X_test_denoised[col+'_Phase']=np.arctan2(np.fft.fft(X_test_denoised[col]).imag, np.fft.fft(X_test_denoised[col]).real) 
X_train_denoised.head()

,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,...,angular_velocity_Y_Amp,angular_velocity_Y_Phase,angular_velocity_Z_Amp,angular_velocity_Z_Phase,linear_acceleration_X_Amp,linear_acceleration_X_Phase,linear_acceleration_Y_Amp,linear_acceleration_Y_Phase,linear_acceleration_Z_Amp,linear_acceleration_Z_Phase
0,0_0,0,0,-0.75853,-0.63435,-0.10488,-0.10597,0.064074,0.017639,0.005879,...,0.016675,0.000000,0.038367,3.141593,0.258562,0.000000,5.772935,0.000000,18.729771,3.141593
1,0_1,0,1,-0.75853,-0.63434,-0.10490,-0.10600,0.055530,0.026568,-0.000263,...,0.002681,-1.613060,0.009702,1.575424,0.003197,-0.246583,0.032862,-1.197256,0.008725,-1.269914
2,0_2,0,2,-0.75853,-0.63435,-0.10492,-0.10597,0.033545,0.026603,-0.003403,...,0.001653,-2.996908,0.004939,0.256185,0.005468,-1.900897,0.004532,0.523438,0.001650,2.420690
3,0_3,0,3,-0.75852,-0.63436,-0.10495,-0.10597,0.020209,0.020964,-0.004373,...,0.002491,-0.199721,0.009018,2.903242,0.005125,-2.158967,0.010557,-1.711563,0.005018,-1.351602
4,0_4,0,4,-0.75852,-0.63435,-0.10495,-0.10596,0.025990,0.016635,-0.005075,...,0.001555,1.651569,0.004734,-1.332085,0.003560,1.420007,0.001038,1.302078,0.001414,2.224966


In [6]:
def quaternion_to_euler(df):
    import math
    def Cal_t2(df):
        df['t2']=2.0*(df['orientation_W'] * df['orientation_Y'] + df['orientation_Z'] * df['orientation_X'])
        df['t2']=df['t2'].transform(lambda x:  1.0 if x> +1.0 else x)
        df['t2']=df['t2'].transform(lambda x: -1.0 if x< -1.0 else x)
        return df
        
#     t0 = +2.0 * (w * x + y * z)
#     t1 = +1.0 - 2.0 * (x * x + y * y)
#     Euler_X = math.atan2(t0, t1)

#     t2 = +2.0 * (w * y - z * x)
#     t2 = +1.0 if t2 > +1.0 else t2
#     t2 = -1.0 if t2 < -1.0 else t2
#     Euler_Y = math.asin(t2)

#     t3 = +2.0 * (w * z + x * y)
#     t4 = 
#     Euler_Z = math.atan2(t3, t4)

    df['t0']=2.0 * (df['orientation_W'] * df['orientation_X'] + df['orientation_Y'] * df['orientation_Z'])
    df['t1']=2.0 * (df['orientation_W'] * df['orientation_X'] + df['orientation_Y'] * df['orientation_Z'])
#     df['t2']=df.transform(Cal_t2)
    df['t3']=2.0 * (df['orientation_W'] * df['orientation_Z'] + df['orientation_X'] * df['orientation_Y'])
    df['t4']=1.0 - 2.0 * (df['orientation_Y']**2 + df['orientation_Z']**2)
    df=Cal_t2(df)
    
    df['Euler_X']=np.arctan2(df['t0'], df['t1'])
    df['Euler_Y']=np.arcsin(df['t2'])
    df['Euler_Z']=np.arctan2(df['t3'], df['t4'])
    del df['t0']
    del df['t1']
    del df['t2']
    del df['t3']
    del df['t4']
    return df

def fe(data):
    
    df = pd.DataFrame()
#     data['totl_anglr_vel'] = (data['angular_velocity_X']**2 + data['angular_velocity_Y']**2 + data['angular_velocity_Z']**2)** 0.5
#     data['totl_linr_acc'] = (data['linear_acceleration_X']**2 + data['linear_acceleration_Y']**2 + data['linear_acceleration_Z']**2)**0.5
    data['totl_xyz'] = (data['orientation_X']**2 + data['orientation_Y']**2 + data['orientation_Z']**2)**0.5   
#     data['acc_vs_vel'] = data['totl_linr_acc'] / data['totl_anglr_vel']
    data=quaternion_to_euler(data)
    data['Total_Euler']=(data['Euler_X']**2+data['Euler_Y']**2+data['Euler_Z']**2)**0.5
    data['Norm_x']=data['orientation_X']/data['totl_xyz']**2
    data['Norm_y']=data['orientation_Y']/data['totl_xyz']**2
    data['Norm_z']=data['orientation_Z']/data['totl_xyz']**2
#     print(data.columns)

    def mean_change_of_abs_change(x):
        return np.mean(np.diff(np.abs(np.diff(x))))    
    features=['totl_xyz','Total_Euler','Norm_x','Norm_y','Norm_z','linear_acceleration_X_Amp', 'linear_acceleration_X_Phase','linear_acceleration_Y_Amp', 'linear_acceleration_Y_Phase','linear_acceleration_Z_Amp', 'linear_acceleration_Z_Phase','angular_velocity_X_Amp', 'angular_velocity_X_Phase','angular_velocity_Y_Amp', 'angular_velocity_Y_Phase','angular_velocity_Z_Amp', 'angular_velocity_Z_Phase']
    for col in features:
        df[col + '_mean'] = data.groupby(['series_id'])[col].mean()
        df[col + '_median'] = data.groupby(['series_id'])[col].median()
        df[col + '_max'] = data.groupby(['series_id'])[col].max()
        df[col + '_min'] = data.groupby(['series_id'])[col].min()
        df[col + '_std'] = data.groupby(['series_id'])[col].std()
        df[col + '_range'] = df[col + '_max'] - df[col + '_min']
        df[col + '_maxtoMin'] = df[col + '_max'] / df[col + '_min']
        df[col + '_mean_abs_chg'] = data.groupby(['series_id'])[col].apply(lambda x: np.mean(np.abs(np.diff(x))))
        df[col + '_abs_max'] = data.groupby(['series_id'])[col].apply(lambda x: np.max(np.abs(x)))
        df[col + '_abs_min'] = data.groupby(['series_id'])[col].apply(lambda x: np.min(np.abs(x)))
        df[col + '_abs_avg'] = (df[col + '_abs_min'] + df[col + '_abs_max'])/2
        df[col + '_mean_change_of_abs_change'] = data.groupby('series_id')[col].apply(mean_change_of_abs_change)
        df[col + 'RMS']=data.groupby('series_id')[col].apply(np.sum(lambda x: x**2))
    return df

In [7]:
%%time
train = fe(X_train_denoised)
test = fe(X_test_denoised)

Wall time: 1min 37s


In [8]:
train

,totl_xyz_mean,totl_xyz_median,totl_xyz_max,totl_xyz_min,totl_xyz_std,totl_xyz_range,totl_xyz_maxtoMin,totl_xyz_mean_abs_chg,totl_xyz_abs_max,totl_xyz_abs_min,...,angular_velocity_Z_Phase_min,angular_velocity_Z_Phase_std,angular_velocity_Z_Phase_range,angular_velocity_Z_Phase_maxtoMin,angular_velocity_Z_Phase_mean_abs_chg,angular_velocity_Z_Phase_abs_max,angular_velocity_Z_Phase_abs_min,angular_velocity_Z_Phase_abs_avg,angular_velocity_Z_Phase_mean_change_of_abs_change,angular_velocity_Z_PhaseRMS
series_id,,,,,,,,,,,,,,,,,,,,,
0,0.994316,0.994306,0.994413,0.994253,0.000042,0.000160,1.000161,0.000006,0.994413,0.994253,...,-2.997489,1.806621,6.139081,-1.048075,2.345934,3.141593,0.030713,1.586153,0.004597,9.869604
1,0.989155,0.989150,0.989305,0.988974,0.000077,0.000331,1.000334,0.000016,0.989305,0.988974,...,-3.119481,1.865968,6.259460,-1.006571,2.386253,3.139979,0.009184,1.574582,0.006670,2.481962
2,0.997471,0.997467,0.997515,0.997438,0.000020,0.000078,1.000078,0.000004,0.997515,0.997438,...,-3.109948,1.902965,6.231350,-1.003683,2.248950,3.121402,0.002793,1.562097,-0.004546,0.065631
3,0.989820,0.989793,0.990242,0.989525,0.000197,0.000717,1.000725,0.000028,0.990242,0.989525,...,-3.129182,1.726617,6.221542,-0.988233,1.947011,3.129182,0.031412,1.580297,-0.031134,8.428814
4,0.990645,0.990633,0.990796,0.990524,0.000074,0.000272,1.000275,0.000007,0.990796,0.990524,...,-3.074234,1.828935,6.198585,-1.016302,2.232038,3.124350,0.011952,1.568151,-0.009029,1.774451
5,0.996697,0.996690,0.996775,0.996642,0.000031,0.000132,1.000133,0.000008,0.996775,0.996642,...,-2.977484,1.792269,6.093230,-1.046436,1.971881,3.115746,0.054626,1.585186,0.028609,0.000943
6,0.998655,0.998658,0.998702,0.998604,0.000029,0.000099,1.000099,0.000003,0.998702,0.998604,...,-3.095976,1.859650,6.217821,-1.008356,2.193036,3.121845,0.005749,1.563797,-0.001148,8.121489
7,0.990122,0.990139,0.990307,0.989912,0.000107,0.000395,1.000399,0.000019,0.990307,0.989912,...,-3.012913,1.817046,6.149632,-1.041092,2.199563,3.136719,0.010628,1.573674,0.007656,4.669292
8,0.999953,0.999953,0.999967,0.999937,0.000008,0.000030,1.000030,0.000003,0.999967,0.999937,...,-3.037633,1.830813,6.177183,-1.033551,2.259175,3.139550,0.005732,1.572641,-0.014184,0.746744


In [9]:
le = LabelEncoder()
Y_train['surface'] = le.fit_transform(Y_train['surface'])

In [10]:
train.fillna(0, inplace = True)
test.fillna(0, inplace = True)
train.replace(-np.inf, 0, inplace = True)
train.replace(np.inf, 0, inplace = True)
test.replace(-np.inf, 0, inplace = True)
test.replace(np.inf, 0, inplace = True)

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, StratifiedKFold
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=11)
sub_preds_rf = np.zeros((test.shape[0], 9))
# print(sub_preds_rf.shape,sub_preds_rf)
oof_preds_rf = np.zeros((train.shape[0]))
score = 0
for i, (train_index, test_index) in enumerate(folds.split(train, Y_train['surface'])):
    print('-'*20, i, '-'*20)
    
    clf =  RandomForestClassifier(n_estimators = 100, n_jobs = -1)
    clf.fit(train.iloc[train_index], Y_train['surface'][train_index])
    oof_preds_rf[test_index] = clf.predict(train.iloc[test_index])
    sub_preds_rf += clf.predict_proba(test) / folds.n_splits
    score += clf.score(train.iloc[test_index], Y_train['surface'][test_index])
    print('train data score: ', clf.score(train.iloc[test_index], Y_train['surface'][test_index]))
    importances = clf.feature_importances_
#     print('importances:',importances)
    indices = np.argsort(importances)[::-1]
    print('indices length:',len(indices))
#     print('indices：',indices)
    features = train.columns
#     print('features',features)
#     print('importances after argsort',importances[indices])
    
   

#     hm = 10
#     plt.figure(figsize=(7, 10))
#     plt.title('Feature Importances')
#     plt.barh(range(len(indices[:hm])), importances[indices][:hm][::-1], color='b', align='center')
#     plt.yticks(range(len(indices[:hm])), [features[i] for i in indices][::-1])
#     plt.xlabel('Relative Importance')
#     plt.show()

print('Avg Accuracy', score / folds.n_splits)

-------------------- 0 --------------------
train data score:  0.793733681462141
indices length: 221
-------------------- 1 --------------------
train data score:  0.7869281045751634
indices length: 221
-------------------- 2 --------------------
train data score:  0.8057742782152231
indices length: 221
-------------------- 3 --------------------
train data score:  0.7921052631578948
indices length: 221
-------------------- 4 --------------------
train data score:  0.7952443857331571
indices length: 221
Avg Accuracy 0.7947571426287159


In [12]:
# Submission['surface'] = le.inverse_transform(sub_preds_rf.argmax(axis=1))
# Submission.to_csv('.\\Kaggle\\CareerCon 2019 - Help Navigate Robots\\data\\Submission_fft.csv', index=False)
# Submission.head(10)

In [13]:
#xgboost
from xgboost import XGBClassifier
sub_preds_xgboost = np.zeros((test.shape[0], 9))
oof_preds_xgboost = np.zeros((train.shape[0]))
score = 0
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, Y_train['surface'])):
    xgb_clf =  XGBClassifier(n_jobs = -1)
    xgb_clf.fit(train.iloc[trn_idx], Y_train['surface'][trn_idx])
    oof_preds_xgboost[val_idx] = xgb_clf.predict(train.iloc[val_idx])
    sub_preds_xgboost += xgb_clf.predict_proba(test) / folds.n_splits
    score += xgb_clf.score(train.iloc[val_idx], Y_train['surface'][val_idx])
    print('Fold: {} score: {}'.format(fold_,xgb_clf.score(train.iloc[val_idx], Y_train['surface'][val_idx])))
print('Avg Accuracy', score / folds.n_splits)
y_test_pred_final = np.array(sub_preds_rf.argmax(axis=1)*0.5+sub_preds_xgboost.argmax(axis=1)*0.5)
y_test_pred_final = np.array([int(val) for val in y_test_pred_final])


Fold: 0 score: 0.7819843342036553
Fold: 1 score: 0.7594771241830065
Fold: 2 score: 0.7585301837270341
Fold: 3 score: 0.7894736842105263
Fold: 4 score: 0.7701453104359313
Avg Accuracy 0.7719221273520308


In [14]:
y_test=Submission.copy()
y_test['surface'] = le.inverse_transform(y_test_pred_final)
y_test.to_csv('.\\Kaggle\\CareerCon 2019 - Help Navigate Robots\\data\\submission_fft.csv', index=False)

In [15]:
# import lightgbm as lgb

hello world
